In [1]:
import tensorflow as tf
import os
import random
import numpy as np

SEED = 614

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

2023-05-16 23:21:47.386281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 23:21:47.487228: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 23:21:48.006359: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-16 23:21:48.006416: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


2023-05-16 23:21:48.746691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2006] Ignoring visible gpu device (device: 1, name: Quadro P1000, pci bus id: 0000:65:00.0, compute capability: 6.1) with core count: 5. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-05-16 23:21:48.747390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 23:21:49.203391: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8454 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


In [3]:
#import os


EPOCHS = 1000
BATCH_SIZE = 8
ARCH = "flor"

IMG_SIZE = (128,32, 1)
DATA_ROOT_PATH = "../data"
IMAGES_PATH = os.path.join(DATA_ROOT_PATH, "TeluguSeg")
IMAGES_PATH

'../data/TeluguSeg'

In [4]:
vocab = None

with open(os.path.join(DATA_ROOT_PATH, "telugu_vocab.txt"),encoding="utf-8") as f:
  vocab = f.readlines()

idx_to_vocab = {i:value.strip() for i, value in enumerate(vocab)}
vocab_to_idx = {value:key for key, value in idx_to_vocab.items()}

In [5]:
train_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_traint.txt"), encoding="utf-8") as f:
  train_data = f.readlines()

In [6]:
valid_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_valt.txt"), encoding="utf-8") as f:
  valid_data = f.readlines()

In [7]:
test_data1 = None

with open(os.path.join(DATA_ROOT_PATH, "new_testt.txt"), encoding="utf-8") as f:
  test_data1 = f.readlines()

In [1]:
charl = None

with open( "charList2.txt", encoding="utf-8") as f:
  charl = f.readlines()

In [2]:
charl=charl[0]

In [3]:
charl[9]

'ఊ'

In [11]:
len(test_data1)

17910

In [12]:
len(train_data)

80693

In [13]:
len(valid_data)

20048

In [14]:
for i in range(4000):
    train_data.append(valid_data[i])

# train data labels

# train data generation

In [15]:

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp

[ WARN:0@0.008] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../data/images/data/10/259/10.jpg'): can't open/read file: check file path/integrity


In [16]:

##label generator
def label_g(l):
    label=idx_to_vocab[l]
    z=[]
    for j in range(27):
        z.append(0)
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z[k]=r
    return z   

In [17]:

##label generator
def label_g1(l):
    label=idx_to_vocab[l]
    z=[]
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z.append(r)
    return z   

In [18]:
train_data[0]

'TeluguSeg/train/7/296/20.jpg 5141\n'

In [19]:
import cv2
import numpy as np

In [20]:
s=np.zeros((8, 27))
s

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp
import cv2

class DataGen(tf.keras.utils.Sequence):
    def __init__(self, data, img_size, batch_size, mode="TRAIN"):
        self.data = data
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        
    def __getitem__(self, i):
        start = i * self.batch_size
        end = (i+1) * self.batch_size
        batch_images = np.zeros((self.batch_size, self.img_size[0], self.img_size[1], 1))
        batch_labels = np.zeros((self.batch_size, 27))
        #batch_labels =[[],[],[],[],[],[],[],[]]
        
        for ii, df_index in enumerate(range(start, end)):
            curr_data = self.data[df_index].split()
            curr_img_path = curr_data[0]
            #curr_label = idx_to_vocab[int(curr_data[1])]
            curr_label = label_g(int(curr_data[1]))

            curr_img_path = "/".join(curr_img_path.split("/")[2:])
            curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)

            
            curr_img = pp.preprocess(curr_img_path, self.img_size)
            curr_img=cv2.adaptiveThreshold(curr_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 199, 5)
            #curr_img = tf.image.resize(curr_img, (self.img_size[0], self.img_size[1]), method="nearest")
            #curr_img = curr_img.numpy().reshape((self.img_size[0], self.img_size[1],1))
            batch_images[ii, :, :,0] = curr_img 
            batch_labels[ii,:] = curr_label
                    
        if self.mode == "TRAIN":
          return batch_images, batch_labels
        else:
          return batch_images
        
    def __len__(self):
      return len(self.data) // self.batch_size

In [ ]:
curr_data = train_data[0].split()
curr_img_path = curr_data[0]
#curr_label = idx_to_vocab[int(curr_data[1])]
curr_label = label_g(int(curr_data[1]))

curr_img_path = "/".join(curr_img_path.split("/")[2:])
curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im

In [6]:
plt.imshow(curr_img_path)

NameError: name 'curr_img_path' is not defined

In [22]:
train_datagen = DataGen(train_data, IMG_SIZE, 8)

In [23]:
valid_datagen = DataGen(valid_data, IMG_SIZE, 8)

In [24]:
test_datagen = DataGen(test_data1, IMG_SIZE, 10000)

In [25]:
train_datagen[0][1]==train_datagen[1][1]

array([[False, False, False, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True, False, False, False, False, False, False,
        False,  True,  True,  True,  True,  True,  True,  True,  True,
  

# model


In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.constraints import MaxNorm

from network.layers import FullGatedConv2D, GatedConv2D, OctConv2D
from tensorflow.keras.layers import Conv2D, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.layers import Input, Add, Activation, Lambda, MaxPooling2D, Reshape

In [27]:
from datetime import datetime

In [28]:
output_path = os.path.join("..", "output", ARCH,str(datetime.now()))
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")

In [29]:
datetime.now()

datetime.datetime(2023, 5, 16, 23, 21, 49, 626891)

In [30]:

from network.model import HTRModel
# create and compile HTRModel
model = HTRModel(architecture=ARCH,
                 input_size=IMG_SIZE,
                 vocab_size=87,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15,
                 reduce_factor=0.1)

#model.compile(learning_rate=0.00555555555                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
model.compile(learning_rate=0.0001)
model.summary(output_path, "summary.txt")

callbacks = model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

/home/project/anaconda3/envs/htr2/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


input size: KerasTensor(type_spec=TensorSpec(shape=(None, 128, 32, 1), dtype=tf.float32, name='input'), name='input', description="created by layer 'input'")
output size: KerasTensor(type_spec=TensorSpec(shape=(None, 16, 143), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 32, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 64, 16, 16)        160       
                                                                 
 p_re_lu (PReLU)             (None, 64, 16, 16)        16        
                                                                 
 batch_normalization (BatchN  (None, 64, 16, 16)       112       
 ormalization)                                                   


In [31]:
start_time = datetime.now()

model.fit(x=train_datagen,
              epochs=EPOCHS,
          validation_data=valid_datagen,
              callbacks=callbacks,
              verbose=1)
total_time = datetime.now() - start_time

print("total time",total_time)

Epoch 1/1000


2023-05-16 23:21:57.692090: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-05-16 23:21:59.320536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8901
2023-05-16 23:21:59.733618: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-16 23:22:00.076042: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fc8d77d4e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-16 23:22:00.076066: I tensorflow/compiler/xla/servi

2023-05-16 23:22:04.183242: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-16 23:22:04.412772: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


  267/10586 [..............................] - ETA: 23:44 - loss: inf   

2023-05-16 23:22:41.414389: W ./tensorflow/core/util/ctc/ctc_loss_calculator.h:499] No valid path found.


...

2023-05-16 23:55:03.453888: W ./tensorflow/core/util/ctc/ctc_loss_calculator.h:499] No valid path found.


 2087/10586 [====>.........................] - ETA: 19:42 - loss: inf

2023-05-16 23:56:19.589623: W ./tensorflow/core/util/ctc/ctc_loss_calculator.h:499] No valid path found.


 2522/10586 [======>.......................] - ETA: 18:39 - loss: inf

KeyboardInterrupt: 

In [ ]:
xt=test_datagen[0][0]
yt=test_datagen[0][1]

In [ ]:
predicts, _ = model.predict(x=xt,
                            ctc_decode=True,
                            verbose=1)

In [ ]:
len(predicts[0])

In [ ]:
predicts[1][0]

In [ ]:
def convert(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i][0])):
            b.append(charl[predicts[i][0][j]])
        a.append(b)
    return a

In [ ]:
def convert1(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i])):
            b.append(charl[int(predicts[i][j])])
        a.append(b)
    return a

In [ ]:
def trim(yt):
    yt1=[]
    for i in range(len(yt)):
        k=0
        for j in range(26,0,-1):
            if yt[i][j]!=0:
                yt1.append(yt[i][0:j+1])
                k=1
                break
        if k==0:
            yt1.append([0])
    return yt1

In [ ]:
yt1=trim(yt)

In [ ]:
len(yt1[0])

In [ ]:
len(yt[0])

In [ ]:
yt1[0][1]

In [ ]:
predicts1=convert(predicts)

In [ ]:
gt=convert1(yt1)

In [ ]:
from data import evaluation

In [ ]:

import string
import unicodedata
import editdistance
import numpy as np


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):
        '''pd, gt = pd.lower(), gt.lower()

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))
	'''
        pd_cer, gt_cer = list(pd), list(gt)
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))
        '''
        pd_wer, gt_wer = pd, gt
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))
        
        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))
        '''
    metrics = [cer]
    metrics = np.mean(metrics, axis=1)

    return metrics

In [ ]:
file5=open(os.path.join(output_path, "cer.txt"),'w')

In [ ]:
evaluate = ocr_metrics(predicts=predicts1,
                                  ground_truth=gt,)
 
print("Calculate Character Error Rate {} ".format(evaluate[0],))
file5.write(str(evaluate[0]))
file5.close()

In [ ]:
predicts1[0]

In [ ]:
predict_path = os.path.join(output_path, "predict")

In [ ]:
ff1=open(os.path.join(output_path,"total_time.txt"),'w')
ff1.write(str(total_time))
ff1.close()

In [ ]:
ground_t_p=os.path.join(output_path, "predict",ARCH)

In [ ]:
ground_t_p

In [ ]:
file1=open(os.path.join(output_path,'predict_ground'),'w')

In [ ]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])

In [ ]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])

In [ ]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])
    file1.write('\n')
    for j in range(len(predicts1[i])):
        file1.write(predicts1[i][j])
    file1.write('\n')

In [ ]:
file1.close()

In [ ]:
file2=open(os.path.join(output_path,'ground_t'),'w')

In [ ]:
for i in range(len(gt)):
    for j in range(len(gt[i])):
        file2.write(gt[i][j])
    file2.write('\n')

In [ ]:
file2.close()

In [ ]:
file3=open('flor1predict1','w')
file3=open(os.path.join(output_path,'predict1'),'w')

In [ ]:
for i in range(len(predicts1)):
    for j in range(len(predicts1[i])):
        file3.write(predicts1[i][j])
    file3.write('\n')

In [ ]:
file3.close()

In [ ]:
import string
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import string
import numpy as np



In [ ]:

def alg4(word1,word2):
  M=[[float('inf')]*(len(word2)+1) for i in range(len(word1)+1)]
          
  #filling last row
  for i in range(len(word2)+1):
    M[len(word1)][i]=len(word2)-i
          
  #filling last column
  for j in range(len(word1)+1):
    M[j][len(word2)]=len(word1)-j
              
  #filling bottom to up manner
          
  for i in range(len(word1)-1,-1,-1):
    for j in range(len(word2)-1,-1,-1):
      if word1[i]==word2[j]:
        M[i][j]=M[i+1][j+1]
      else:
        M[i][j]=1+min(M[i+1][j],M[i][j+1],M[i+1][j+1])

  x,y=0,0
  #print(x,y)
  count=0
  while x<len(M)-1 and y<len(M[0])-1:
    current=M[x][y]
    dia=M[x+1][y+1]
    right=M[x][y+1]
    bottom=M[x+1][y]
    if dia<=right and dia<=bottom and dia<=current:
      if dia==current-1:
        print("Substitution-->",word1[x],"replaced by",word2[y])
        #array[string.printable[:95].find(word1[x]),string.printable[:95].find(word2[y])]=array[string.printable[:95].find(word1[x]),string.printable[:95].find(word2[y])]+1
        sub[dict1[word1[x]]]=sub[dict1[word1[x]]]+1
        subs[dict1[word2[x]]][dict1[word1[x]]]=subs[dict1[word2[x]]][dict1[word1[x]]]+1
        count=count+1
        x=x+1
        y=y+1
      else:
        print("No operation-->",word1[x])
        x=x+1
        y=y+1
      
    elif right<=bottom and right<=current:
      print("Insertion",word2[y])
      ins[dict1[word2[y]]]=ins[dict1[word2[y]]]+1

      count=count+1
      y=y+1
    else:
      print("Deletion",word1[x])
      delete[dict1[word1[x]]]=delete[dict1[word1[x]]]+1
      x=x+1
      count=count+1
  print("total operations",count)
  

In [ ]:
delete=[]
ins=[]
sub=[]

for i in range(len(charl)+1):
    delete.append(0)
    ins.append(0)
    sub.append(0)

In [ ]:
file1=open(os.path.join(output_path,'1delete.txt'),'w')
file2=open(os.path.join(output_path,'1ins.txt'),'w')
file3=open(os.path.join(output_path,'1sub.txt'),'w')

In [ ]:
for i in range(len(charl)):
    print(charl[i])
    file1.write(charl[i])
    file1.write('\n')
    file1.write(str(delete[i]))
    file1.write('\n')
    file2.write(charl[i])
    file2.write('\n')
    file2.write(str(ins[i]))
    file2.write('\n')
    file3.write(charl[i])
    file3.write('\n')
    file3.write(str(sub[i]))
    file3.write('\n')
    

In [ ]:
file1.close()
file2.close()
file3.close()

In [ ]:
subs=[]
for i in range(len(charl)):
    z=[]
    for j in range(len(charl)):
        z.append(0)
    subs.append(z)

In [ ]:
for i in range(len(charl)):
    
    for j in range(len(charl)):
        if subs[i][j]!=0:
            print(subs[i][j])

In [ ]:
len(charl)

In [ ]:
dict1={}
for i in range(len(charl)):
    dict1[charl[i]]=i

In [ ]:
dict1

In [ ]:
s1=""
for j in range(len(gt[70])):
    s1=s1+gt[70][j]

In [ ]:
s2=""
for j in range(len(predicts1[70])):
    s2=s2+predicts1[70][j]

In [ ]:
alg4(s1,s2)

In [ ]:
file4=open('flor1subsc.txt','w')
file4=open(os.path.join(output_path,'1subsc.txt'),'w')

In [ ]:
for i in range(len(subs)):
    for j in range(len(subs)):
        file4.write(str(subs[i][j]))
        file4.write(" ")
    file4.write('\n')

In [ ]:
file4.close()

In [ ]:
gt[70]

In [ ]:
predicts1[70]

# charlist generation


In [ ]:
str(total_time)

In [ ]:
output_path